In [3]:
import crocoddyl
import pinocchio
import numpy as np
import example_robot_data
from crocoddyl.utils.biped import SimpleBipedGaitProblem
from pinocchio.robot_wrapper import RobotWrapper

crocoddyl.switchToNumpyMatrix()

# Creating the lower-body part of Talos
#talos_legs = example_robot_data.loadTalosLegs()

# Loading the RH5 Model
URDF_FILENAME = "RH5_Lower_Body_FixedMeshPaths.urdf"
URDF_SUBPATH = "/abstract-urdf/urdf/" + URDF_FILENAME
modelPath = "/home/dfki.uni-bremen.de/jesser/Dev/rh5-models"

# Fix mesh file paths by parsing urdf file

rh5_legs = RobotWrapper.BuildFromURDF(modelPath + URDF_SUBPATH, [modelPath], pinocchio.JointModelFreeFlyer()) # Load URDF file
#readParamsFromSrdf(robot, modelPath + SRDF_SUBPATH, False) # Load SRDF file
#assert ((robot.model.armature[:6] == 0.).all()) 
#example_robot_data.addFreeFlyerJointLimits(rh5_legs) # Add the free-flyer joint limits

# Setting up the 3d walking problem
rightFoot = 'LL_AnkleFT'
leftFoot = 'LR_AnkleFT'

#Display the loaded robot model
display = crocoddyl.GepettoDisplay(rh5_legs, 4, 4, frameNames=[rightFoot, leftFoot])

In [4]:
gait = SimpleBipedGaitProblem(rh5_legs.model, rightFoot, leftFoot)

# Create the initial state
q0 = rh5_legs.q0.copy()
v0 = pinocchio.utils.zero(rh5_legs.model.nv)
x0 = np.concatenate([q0,v0])


# Creating the walking problem
stepLength = 0.6 # meters
stepHeight = 0.1 # meters
timeStep = 0.0375 # seconds
stepKnots = 20
supportKnots = 10
problem = gait.createWalkingProblem(x0, stepLength, stepHeight, timeStep, stepKnots, supportKnots)


# Solving the 3d walking problem using Feasibility-prone DDP
ddp = crocoddyl.SolverFDDP(problem)
display = crocoddyl.GepettoDisplay(rh5_legs, 4, 4, frameNames=[rightFoot, leftFoot])
cameraTF = [3., 3.68, 0.84, 0.2, 0.62, 0.72, 0.22]
ddp.setCallbacks([crocoddyl.CallbackLogger(),
                  crocoddyl.CallbackVerbose(),
                  crocoddyl.CallbackDisplay(display)])
ddp.th_stop = 1e-9
init_xs = [rh5_legs.model.defaultState] * (problem.T + 1)
init_us = []
maxiter = 1000
regInit = 0.1
ddp.solve(init_xs, init_us, maxiter, False, regInit)

KeyError: 'Invalid key'

With the following commands we can plot 
 - the state and control trajectories, and
 - the DDP performance

In [ ]:
%matplotlib inline

# Plotting the solution and the DDP convergence
log = ddp.getCallbacks()[0]
crocoddyl.plotOCSolution(log.xs, log.us)
crocoddyl.plotConvergence(log.costs, log.u_regs, log.x_regs, log.grads, log.stops, log.steps)


Finally we can visualize the solution using gepetto-viewer.

In [ ]:
# Visualization of the DDP solution in gepetto-viewer
#rate = 2.
#display.displayFromSolver(ddp, rate)

display = crocoddyl.GepettoDisplay(rh5_legs, frameNames=[rightFoot, leftFoot])
display.displayFromSolver(ddp)

## VI. Understanding the walking problem

In this problem we pre-defined a 20 and 10 knots for the step and double support phases with $dt=$3.75e-2, repectively. 

 1. Could you tell us how much is the foot step and double support duration?
 2. What happens when do we change $dt$ (e.g. 2e-2 secs)?
 3. What happens when do we change the number of step knots (e.g. 10)?
